In [1]:
#Authors: Akbota Assan (A69037121), Devana Perupurayil (A69034326), Melissa Medina-Peregrina (A59016508)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import emcee

# Load CSV files into DataFrames
DetectorA = pd.read_csv('DetectorA.csv')
DetectorB = pd.read_csv('DetectorB.csv')
DetectorC = pd.read_csv('DetectorC.csv')
DetectorTarget = pd.read_csv('DetectorTarget.csv')

# Define parameters
energy_bins = np.linspace(1400, 2200, 50)
data_target, _ = np.histogram(DetectorTarget['energy'], bins=energy_bins)
dataA, _ = np.histogram(DetectorA['energy'], bins=energy_bins)
dataB, _ = np.histogram(DetectorB['energy'], bins=energy_bins)
dataC, _ = np.histogram(DetectorC['energy'], bins=energy_bins)

# Priors for θA, θB, θC, and θNLDBD (Gaussian)
prior_thetaA = scipy.stats.norm(loc=1350, scale=100)
prior_thetaB = scipy.stats.norm(loc=770, scale=270)
prior_thetaC = scipy.stats.norm(loc=1500, scale=500)
prior_thetaNLDBD = scipy.stats.norm(loc=2039, scale=1)

# Define Likelihood Function
def model(thetaA, thetaB, thetaC):
    return thetaA * dataA + thetaB * dataB + thetaC * dataC

def log_likelihood(params):
    thetaA, thetaB, thetaC = params
    if thetaA < 0 or thetaB < 0 or thetaC < 0:
        return -np.inf
    model_counts = np.maximum(model(thetaA, thetaB, thetaC), 1e-6)
    likelihood = scipy.stats.poisson.logpmf(data_target, model_counts)
    return np.sum(likelihood)

def log_prior(params):
    thetaA, thetaB, thetaC = params
    if thetaA < 0 or thetaB < 0 or thetaC < 0:
        return -np.inf
    return np.log(prior_thetaA.pdf(thetaA)) + np.log(prior_thetaB.pdf(thetaB)) + np.log(prior_thetaC.pdf(thetaC))

def log_posterior(thetaA, thetaB, thetaC):
    params = (thetaA, thetaB, thetaC)
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params)


In [ ]:
import functools
import inspect
import emcee

def wrap(func, **kwargs):
    fixed = {}
    names, values, lnpriors = [], [], []
    funcsig = inspect.signature(func)
    try:
        funcargs = {name: kwargs[name] for name in funcsig.parameters}
    except KeyError:
        raise ValueError('Missing arguments.')
    bound = funcsig.bind(**funcargs)
    bound.apply_defaults()
    NoPrior = lambda x: 0.
    for name, value in bound.arguments.items():
        if isinstance(value, list):
            names.append(name)
            values.append(value.pop(0))
            lnpriors.append(value.pop(0) if value else NoPrior)
            if value:
                raise ValueError('Invalid syntax for argument {}.'.format(name))
        else:
            fixed[name] = value
    partial = functools.partial(func, **fixed)
    def wrapped(theta):
        if len(theta) != len(names):
            raise ValueError('expected list of {} values.'.format(len(names)))
        result = 0.
        for lnprior, value in zip(lnpriors, theta):
            result += lnprior(value)
            if not np.isfinite(result):
                # theta is not allowed by this prior.
                return -np.inf
        args = dict(zip(names, theta))
        result += partial(**args)
        return result
    # Remove function args from kwargs.
    for name in funcargs:
        kwargs.pop(name, None)
    return wrapped, names, values, kwargs


def sample(func, names, values, nwalkers=20, nsamples=1000, abs_rms=1e-4,
           frac_rms=1e-3, burnin=100, random_state=None):
    
    if random_state is None:
        random_state = np.random.RandomState()
    # Generate sampler starting points.
    ndim = len(names)
    values = np.array(values, float)
    initial = np.tile(values, (nwalkers, 1))
    rms = np.maximum(abs_rms, frac_rms * values)
    initial += rms * random_state.normal(size=(nwalkers, ndim))
    # Initialize and run sampler.
    sampler = emcee.EnsembleSampler(nwalkers, ndim, func)
    n_per_chain = 1 + nsamples // nwalkers + burnin
    sampler.run_mcmc(initial, n_per_chain, rstate0=random_state.get_state())
    # Remove burnin and return results in a DataFrame.
    chain = sampler.chain[:, burnin:].reshape(-1, ndim)[:nsamples]
    return pd.DataFrame(chain, columns=names)


def MCMC_sample(func, **kwargs):
    # Wrap the input function.
    wrapped, names, values, kwargs = wrap(func, **kwargs)
    # Generate emcee samples.
    return sample(wrapped, names, values, **kwargs)

In [ ]:
results = MCMC_sample(log_posterior, thetaA=[1350, prior_thetaA.logpdf], thetaB=[770, prior_thetaB.logpdf], thetaC=[1500, prior_thetaC.logpdf])
print(results.describe())